In [1]:
import zipfile

In [2]:
with zipfile.ZipFile("./zippedData/im.db.zip", 'r') as zip_ref:
    zip_ref.extractall("./zippedData/")

In [101]:
import pandas as pd
import sqlite3
import pandasql
import numpy as np

conn = sqlite3.connect("./zippedData/im.db")
cur = conn.cursor()

In [4]:
pd.read_sql(
'''
SELECT *
FROM sqlite_master
''', 
conn)

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


In [5]:
q1 = pd.read_sql(
'''
SELECT 
    *
FROM 
    movie_basics
ORDER BY
    start_year ASC
''', 
conn)

In [6]:
q1.head(15) # movie basics is a table showing all movies from 2010- and gives runtime and genres

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0146592,Pál Adrienn,Pál Adrienn,2010,136.0,Drama
1,tt0154039,So Much for Justice!,Oda az igazság,2010,100.0,History
2,tt0162942,Children of the Green Dragon,A zöld sárkány gyermekei,2010,89.0,Drama
3,tt0230212,The Final Journey,The Final Journey,2010,120.0,Drama
4,tt0312305,Quantum Quest: A Cassini Space Odyssey,Quantum Quest: A Cassini Space Odyssey,2010,45.0,"Adventure,Animation,Sci-Fi"
5,tt0326592,The Overnight,The Overnight,2010,88.0,None
6,tt0326965,In My Sleep,In My Sleep,2010,104.0,"Drama,Mystery,Thriller"
7,tt0331312,This Wretched Life,This Wretched Life,2010,99.0,"Comedy,Drama"
8,tt0337882,Blind Sided,Blind Sided,2010,NaN,"Comedy,Crime,Drama"
9,tt0378546,The Payback Man,The Payback Man,2010,NaN,"Crime,Drama,Thriller"


q1 = pd.read_sql(
'''
SELECT 
    principals.*,
    per.primary_name
FROM 
    principals
    INNER JOIN persons AS per ON principals.person_id = per.person_id
WHERE
    per.primary_name == "Hans Zimmer" 
   
''', 
conn)

q1

In [7]:
# Decided to also filter 'composer' due to it 'composer' being the main composer of the movie as 
# opposed to 'head composer', 'main composer', 'lead composer', etc. 
movie_info = pd.read_sql(
'''
SELECT 
    mb.movie_id,
    mb.primary_title,
    mb.original_title,
    mb.start_year,
    mb.genres,
    mr.averagerating AS average_rating,
    mr.numvotes AS num_votes,
    per.primary_name AS persons_name,
    princ.category AS persons_job
FROM 
    movie_basics AS mb
    INNER JOIN movie_ratings AS mr ON mb.movie_id = mr.movie_id
    INNER JOIN principals AS princ ON mb.movie_id = princ.movie_id
    INNER JOIN persons AS per ON princ.person_id = per.person_id
    INNER JOIN movie_akas AS ma ON mb.movie_id = ma.movie_id
WHERE
    princ.category IN ('actor', 'actress', 'director', 'writer', 'producer', 'composer', 'director of photography')
    AND ma.region == 'US'
ORDER BY
    mb.start_year ASC
''', 
conn)

In [8]:
movie_info

,movie_id,primary_title,original_title,start_year,genres,average_rating,num_votes,persons_name,persons_job
0,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Johnny Depp,actor
1,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Helena Bonham Carter,actress
2,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Tim Burton,director
3,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Anne Hathaway,actress
4,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Joe Roth,producer
...,...,...,...,...,...,...,...,...,...
214270,tt9562694,Alien Warfare,Alien Warfare,2019,"Action,Sci-Fi",2.6,1509,Scott C. Roe,actor
214271,tt9562694,Alien Warfare,Alien Warfare,2019,"Action,Sci-Fi",2.6,1509,David Meadows,actor
214272,tt9562694,Alien Warfare,Alien Warfare,2019,"Action,Sci-Fi",2.6,1509,Nathan Zoebl,writer
214273,tt9562694,Alien Warfare,Alien Warfare,2019,"Action,Sci-Fi",2.6,1509,Ben Bailey,writer


In [9]:
q1 = pd.read_sql(
'''
SELECT 
    *
FROM 
    directors
''', 
conn)

In [10]:
q1.head()

,movie_id,person_id
0,tt0285252,nm0899854
1,tt0462036,nm1940585
2,tt0835418,nm0151540
3,tt0835418,nm0151540
4,tt0878654,nm0089502


In [11]:
q1 = pd.read_sql(
'''
SELECT 
    *
FROM 
    known_for
''', 
conn)

In [12]:
q1.head(10)

,person_id,movie_id
0,nm0061671,tt0837562
1,nm0061671,tt2398241
2,nm0061671,tt0844471
3,nm0061671,tt0118553
4,nm0061865,tt0896534
5,nm0061865,tt6791238
6,nm0061865,tt0287072
7,nm0061865,tt1682940
8,nm0062070,tt1470654
9,nm0062070,tt0363631


In [13]:
q1 = pd.read_sql(
'''
SELECT 
    *
FROM 
    movie_akas

''', 
conn)

In [14]:
q1

,movie_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,None,None,0.0
1,tt0369610,11,Jurashikku warudo,JP,None,imdbDisplay,None,0.0
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,None,imdbDisplay,None,0.0
3,tt0369610,13,O Mundo dos Dinossauros,BR,None,None,short title,0.0
4,tt0369610,14,Jurassic World,FR,None,imdbDisplay,None,0.0
...,...,...,...,...,...,...,...,...
331698,tt9827784,2,Sayonara kuchibiru,None,None,original,None,1.0
331699,tt9827784,3,Farewell Song,XWW,en,imdbDisplay,None,0.0
331700,tt9880178,1,La atención,None,None,original,None,1.0
331701,tt9880178,2,La atención,ES,None,None,None,0.0


In [15]:
q1 = pd.read_sql(
'''
SELECT 
    DISTINCT region
FROM 
    movie_akas
WHERE
    region LIKE '%US%'
ORDER BY
    region DESC
''', 
conn)

In [16]:
q1

,region
0,US


In [17]:
q1 = pd.read_sql(
'''
SELECT 
    *
FROM 
    movie_ratings
''', 
conn)

q1

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21
...,...,...,...
73851,tt9805820,8.1,25
73852,tt9844256,7.5,24
73853,tt9851050,4.7,14
73854,tt9886934,7.0,5


In [18]:
q1 = pd.read_sql(
'''
SELECT 
    *
FROM 
    persons
''', 
conn)

q1

,person_id,primary_name,birth_year,death_year,primary_profession
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator"
...,...,...,...,...,...
606643,nm9990381,Susan Grobes,NaN,NaN,actress
606644,nm9990690,Joo Yeon So,NaN,NaN,actress
606645,nm9991320,Madeline Smith,NaN,NaN,actress
606646,nm9991786,Michelle Modigliani,NaN,NaN,producer


In [19]:
q1 = pd.read_sql(
'''
SELECT 
    *
FROM 
    writers
''', 
conn)

q1

,movie_id,person_id
0,tt0285252,nm0899854
1,tt0438973,nm0175726
2,tt0438973,nm1802864
3,tt0462036,nm1940585
4,tt0835418,nm0310087
...,...,...
255868,tt8999892,nm10122246
255869,tt8999974,nm10122357
255870,tt9001390,nm6711477
255871,tt9004986,nm4993825


### Now that we have our movie info dataframe we can look to merge it with the tn.movie_budgets csv

In [20]:
# import our tn.movie_budgets.csv
tn_movie_budgets = pd.read_csv('./zippedData/tn.movie_budgets.csv.gz')

In [21]:
# now let's take a look at it
print(tn_movie_budgets.shape)
tn_movie_budgets.head()

(5782, 6)


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [22]:
# now let's look at our movie_info data frame
print(movie_info.shape)
movie_info.head()

(214275, 9)


,movie_id,primary_title,original_title,start_year,genres,average_rating,num_votes,persons_name,persons_job
0,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Johnny Depp,actor
1,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Helena Bonham Carter,actress
2,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Tim Burton,director
3,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Anne Hathaway,actress
4,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Joe Roth,producer


They have a common movie title column. I'll create a new column in each that is a normalization of the movie titles. They will be lower case and stripped of any trailing white spaces.

In [23]:
x = '   HeLo ThiS is A movIE   '
x.strip().lower()

'helo this is a movie'

In [24]:
# first the tn_movie_budgets
tn_movie_budgets['title_norm'] = tn_movie_budgets['movie'].str.strip().str.lower()

In [25]:
tn_movie_budgets

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,title_norm
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279",avatar
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",pirates of the caribbean: on stranger tides
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350",dark phoenix
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963",avengers: age of ultron
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747",star wars ep. viii: the last jedi
...,...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0,red 11
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495",following
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338",return to the land of wonders
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0,a plague so pleasant


In [26]:
tn_movie_budgets.loc[tn_movie_budgets['movie'] == 'Alice in Wonderland']

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,title_norm
50,51,"Mar 5, 2010",Alice in Wonderland,"$200,000,000","$334,191,110","$1,025,491,110",alice in wonderland
4759,60,"Jul 28, 1951",Alice in Wonderland,"$3,000,000",$0,$0,alice in wonderland


In [27]:
# now the movie_info dataframe
movie_info['title_norm'] = movie_info['primary_title'].str.strip().str.lower()

In [295]:
movie_info.head()

,movie_id,primary_title,original_title,start_year,genres,average_rating,num_votes,persons_name,persons_job,title_norm
0,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Johnny Depp,actor,alice in wonderland 2010
1,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Helena Bonham Carter,actress,alice in wonderland 2010
2,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Tim Burton,director,alice in wonderland 2010
3,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Anne Hathaway,actress,alice in wonderland 2010
4,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Joe Roth,producer,alice in wonderland 2010


In [293]:
pd.reset_option("all") 


: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.



C:\Users\ghall\anaconda3\envs\learn-env\lib\site-packages\pandas\_config\config.py:622: FutureWarning: 
: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.

  warnings.warn(d.msg, FutureWarning)


In [294]:
movie_info.groupby(by=['movie_id', 'primary_title']).count().sort_values(by='primary_title')

,,original_title,start_year,genres,average_rating,num_votes,persons_name,persons_job,title_norm
movie_id,primary_title,,,,,,,,
tt2346170,#1 Serial Killer,8,8,8,8,8,8,8,8
tt3120962,#5,2,2,2,2,2,2,2,2
tt5074174,#BeRobin the Movie,1,1,1,1,1,1,1,1
tt6856592,#Captured,9,9,9,9,9,9,9,9
tt5803530,#DigitalLivesMatter,10,10,10,10,10,10,10,10
...,...,...,...,...,...,...,...,...,...
tt6194704,À 2 heures de Paris,6,6,6,6,6,6,6,6
tt3550444,Ækte vare,9,9,9,9,9,9,9,9
tt1822381,Évocateur: The Morton Downey Jr. Movie,4,4,4,4,4,4,4,4


In [261]:
movie_info.loc[movie_info['movie_id'] == 'tt2049386']

,movie_id,primary_title,original_title,start_year,genres,average_rating,num_votes,persons_name,persons_job,title_norm
8978,tt2049386,Alice in Wonderland,Alice in Wonderland,2010,"Fantasy,Musical",6.5,6,James Fotopoulos,director,alice in wonderland 2010
8979,tt2049386,Alice in Wonderland,Alice in Wonderland,2010,"Fantasy,Musical",6.5,6,Lauren Nicole Goode,actress,alice in wonderland 2010
8980,tt2049386,Alice in Wonderland,Alice in Wonderland,2010,"Fantasy,Musical",6.5,6,Nate Archer,composer,alice in wonderland 2010
8981,tt2049386,Alice in Wonderland,Alice in Wonderland,2010,"Fantasy,Musical",6.5,6,Lauren Chapman,producer,alice in wonderland 2010
8982,tt2049386,Alice in Wonderland,Alice in Wonderland,2010,"Fantasy,Musical",6.5,6,Sarah Evans,actress,alice in wonderland 2010
8983,tt2049386,Alice in Wonderland,Alice in Wonderland,2010,"Fantasy,Musical",6.5,6,Ono,composer,alice in wonderland 2010
8984,tt2049386,Alice in Wonderland,Alice in Wonderland,2010,"Fantasy,Musical",6.5,6,Walter Slaughter,composer,alice in wonderland 2010


Now I'll merge them based on the title_norm column

In [284]:
movie_info.merge(tn_movie_budgets, how='inner', on='title_norm',suffixes=('_movie_info', '_movie_budgets'))

,movie_id,primary_title,original_title,start_year,genres,average_rating,num_votes,persons_name,persons_job,title_norm,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Johnny Depp,actor,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,"$200,000,000","$334,191,110","$1,025,491,110"
1,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Helena Bonham Carter,actress,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,"$200,000,000","$334,191,110","$1,025,491,110"
2,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Tim Burton,director,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,"$200,000,000","$334,191,110","$1,025,491,110"
3,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Anne Hathaway,actress,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,"$200,000,000","$334,191,110","$1,025,491,110"
4,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Joe Roth,producer,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,"$200,000,000","$334,191,110","$1,025,491,110"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20419,tt6139732,Aladdin,Aladdin,2019,"Adventure,Comedy,Family",7.4,57549,Dan Lin,producer,aladdin 2019,81,"May 24, 2019",Aladdin,"$182,000,000","$246,734,314","$619,234,314"
20420,tt6139732,Aladdin,Aladdin,2019,"Adventure,Comedy,Family",7.4,57549,Jonathan Eirich,producer,aladdin 2019,81,"May 24, 2019",Aladdin,"$182,000,000","$246,734,314","$619,234,314"
20421,tt6139732,Aladdin,Aladdin,2019,"Adventure,Comedy,Family",7.4,57549,Marwan Kenzari,actor,aladdin 2019,81,"May 24, 2019",Aladdin,"$182,000,000","$246,734,314","$619,234,314"
20422,tt6139732,Aladdin,Aladdin,2019,"Adventure,Comedy,Family",7.4,57549,Naomi Scott,actress,aladdin 2019,81,"May 24, 2019",Aladdin,"$182,000,000","$246,734,314","$619,234,314"


The problem here is that movies that share the same name but different years are causing problems. To fix this we can add the year to the title_norm column for each to differentiate movies with same names and different years.

In [30]:
# adding release year to title_norm for tn_movie_budgets
tn_movie_budgets['title_norm'] = tn_movie_budgets['movie'].str.strip().str.lower() + ' ' \
                                    + tn_movie_budgets['release_date'].str[-4:]

In [31]:
tn_movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,title_norm
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279",avatar 2009
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",pirates of the caribbean: on stranger tides 2011
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350",dark phoenix 2019
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963",avengers: age of ultron 2015
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747",star wars ep. viii: the last jedi 2017


In [32]:
movie_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214275 entries, 0 to 214274
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   movie_id        214275 non-null  object 
 1   primary_title   214275 non-null  object 
 2   original_title  214275 non-null  object 
 3   start_year      214275 non-null  int64  
 4   genres          213612 non-null  object 
 5   average_rating  214275 non-null  float64
 6   num_votes       214275 non-null  int64  
 7   persons_name    214275 non-null  object 
 8   persons_job     214275 non-null  object 
 9   title_norm      214275 non-null  object 
dtypes: float64(1), int64(2), object(7)
memory usage: 16.3+ MB


In [33]:
# adding release year to title_norm for movie_info
movie_info['title_norm'] = movie_info['primary_title'].str.strip().str.lower() + ' ' \
                            + movie_info['start_year'].astype(str)

In [34]:
movie_info.head()

,movie_id,primary_title,original_title,start_year,genres,average_rating,num_votes,persons_name,persons_job,title_norm
0,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Johnny Depp,actor,alice in wonderland 2010
1,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Helena Bonham Carter,actress,alice in wonderland 2010
2,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Tim Burton,director,alice in wonderland 2010
3,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Anne Hathaway,actress,alice in wonderland 2010
4,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Joe Roth,producer,alice in wonderland 2010


title_norm worked for both now we'll try to merge the tables again

In [35]:
df_movie_info_budget = movie_info.merge(tn_movie_budgets, 
                 how='inner', 
                 on='title_norm', 
                 suffixes=('_movie_info', '_movie_budgets'))

In [36]:
# The merged looked like it worked now it's time to take a look at the new dataframe
df_movie_info_budget.head(10)

,movie_id,primary_title,original_title,start_year,genres,average_rating,num_votes,persons_name,persons_job,title_norm,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Johnny Depp,actor,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,"$200,000,000","$334,191,110","$1,025,491,110"
1,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Helena Bonham Carter,actress,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,"$200,000,000","$334,191,110","$1,025,491,110"
2,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Tim Burton,director,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,"$200,000,000","$334,191,110","$1,025,491,110"
3,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Anne Hathaway,actress,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,"$200,000,000","$334,191,110","$1,025,491,110"
4,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Joe Roth,producer,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,"$200,000,000","$334,191,110","$1,025,491,110"
5,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Lewis Carroll,writer,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,"$200,000,000","$334,191,110","$1,025,491,110"
6,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Jennifer Todd,producer,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,"$200,000,000","$334,191,110","$1,025,491,110"
7,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Suzanne Todd,producer,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,"$200,000,000","$334,191,110","$1,025,491,110"
8,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Linda Woolverton,writer,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,"$200,000,000","$334,191,110","$1,025,491,110"
9,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Mia Wasikowska,actress,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,"$200,000,000","$334,191,110","$1,025,491,110"


In [37]:
df_movie_info_budget.tail(10)

,movie_id,primary_title,original_title,start_year,genres,average_rating,num_votes,persons_name,persons_job,title_norm,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
20414,tt6139732,Aladdin,Aladdin,2019,"Adventure,Comedy,Family",7.4,57549,Mena Massoud,actor,aladdin 2019,81,"May 24, 2019",Aladdin,"$182,000,000","$246,734,314","$619,234,314"
20415,tt6139732,Aladdin,Aladdin,2019,"Adventure,Comedy,Family",7.4,57549,Will Smith,actor,aladdin 2019,81,"May 24, 2019",Aladdin,"$182,000,000","$246,734,314","$619,234,314"
20416,tt6139732,Aladdin,Aladdin,2019,"Adventure,Comedy,Family",7.4,57549,Guy Ritchie,director,aladdin 2019,81,"May 24, 2019",Aladdin,"$182,000,000","$246,734,314","$619,234,314"
20417,tt6139732,Aladdin,Aladdin,2019,"Adventure,Comedy,Family",7.4,57549,John August,writer,aladdin 2019,81,"May 24, 2019",Aladdin,"$182,000,000","$246,734,314","$619,234,314"
20418,tt6139732,Aladdin,Aladdin,2019,"Adventure,Comedy,Family",7.4,57549,Alan Menken,composer,aladdin 2019,81,"May 24, 2019",Aladdin,"$182,000,000","$246,734,314","$619,234,314"
20419,tt6139732,Aladdin,Aladdin,2019,"Adventure,Comedy,Family",7.4,57549,Dan Lin,producer,aladdin 2019,81,"May 24, 2019",Aladdin,"$182,000,000","$246,734,314","$619,234,314"
20420,tt6139732,Aladdin,Aladdin,2019,"Adventure,Comedy,Family",7.4,57549,Jonathan Eirich,producer,aladdin 2019,81,"May 24, 2019",Aladdin,"$182,000,000","$246,734,314","$619,234,314"
20421,tt6139732,Aladdin,Aladdin,2019,"Adventure,Comedy,Family",7.4,57549,Marwan Kenzari,actor,aladdin 2019,81,"May 24, 2019",Aladdin,"$182,000,000","$246,734,314","$619,234,314"
20422,tt6139732,Aladdin,Aladdin,2019,"Adventure,Comedy,Family",7.4,57549,Naomi Scott,actress,aladdin 2019,81,"May 24, 2019",Aladdin,"$182,000,000","$246,734,314","$619,234,314"
20423,tt6139732,Aladdin,Aladdin,2019,"Adventure,Comedy,Family",7.4,57549,Mena Massoud,actor,aladdin 2019,81,"May 24, 2019",Aladdin,"$182,000,000","$246,734,314","$619,234,314"


In [38]:
df_movie_info_budget.duplicated().sum()

7735

In [42]:
df_movie_info_budget[df_movie_info_budget.duplicated()].head(20)

,movie_id,primary_title,original_title,start_year,genres,average_rating,num_votes,persons_name,persons_job,title_norm,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
10,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Johnny Depp,actor,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,"$200,000,000","$334,191,110","$1,025,491,110"
11,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Helena Bonham Carter,actress,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,"$200,000,000","$334,191,110","$1,025,491,110"
12,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Tim Burton,director,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,"$200,000,000","$334,191,110","$1,025,491,110"
13,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Anne Hathaway,actress,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,"$200,000,000","$334,191,110","$1,025,491,110"
14,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Joe Roth,producer,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,"$200,000,000","$334,191,110","$1,025,491,110"
15,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Lewis Carroll,writer,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,"$200,000,000","$334,191,110","$1,025,491,110"
16,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Jennifer Todd,producer,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,"$200,000,000","$334,191,110","$1,025,491,110"
17,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Suzanne Todd,producer,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,"$200,000,000","$334,191,110","$1,025,491,110"
18,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Linda Woolverton,writer,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,"$200,000,000","$334,191,110","$1,025,491,110"
19,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Mia Wasikowska,actress,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,"$200,000,000","$334,191,110","$1,025,491,110"


In [40]:
df_movie_info_budget.loc[(df_movie_info_budget['primary_title'] == 'Alice in Wonderland') \
                         & (df_movie_info_budget['persons_name'] == 'Johnny Depp')]

,movie_id,primary_title,original_title,start_year,genres,average_rating,num_votes,persons_name,persons_job,title_norm,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Johnny Depp,actor,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,"$200,000,000","$334,191,110","$1,025,491,110"
10,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Johnny Depp,actor,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,"$200,000,000","$334,191,110","$1,025,491,110"
20,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Johnny Depp,actor,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,"$200,000,000","$334,191,110","$1,025,491,110"


### There are duplicate rows in the new data frame. I'll modify the data frame to drop the duplicate rows

In [44]:
df_movie_info_budget.drop_duplicates(inplace=True)

In [45]:
df_movie_info_budget.loc[(df_movie_info_budget['primary_title'] == 'Alice in Wonderland') \
                         & (df_movie_info_budget['persons_name'] == 'Johnny Depp')]

,movie_id,primary_title,original_title,start_year,genres,average_rating,num_votes,persons_name,persons_job,title_norm,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Johnny Depp,actor,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,"$200,000,000","$334,191,110","$1,025,491,110"


### Removing the duplicates worked

In [47]:
df_movie_info_budget.head()

,movie_id,primary_title,original_title,start_year,genres,average_rating,num_votes,persons_name,persons_job,title_norm,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Johnny Depp,actor,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,"$200,000,000","$334,191,110","$1,025,491,110"
1,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Helena Bonham Carter,actress,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,"$200,000,000","$334,191,110","$1,025,491,110"
2,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Tim Burton,director,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,"$200,000,000","$334,191,110","$1,025,491,110"
3,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Anne Hathaway,actress,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,"$200,000,000","$334,191,110","$1,025,491,110"
4,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Joe Roth,producer,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,"$200,000,000","$334,191,110","$1,025,491,110"


In [46]:
# look at the info of the dataframe
df_movie_info_budget.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12689 entries, 0 to 20414
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   movie_id           12689 non-null  object 
 1   primary_title      12689 non-null  object 
 2   original_title     12689 non-null  object 
 3   start_year         12689 non-null  int64  
 4   genres             12689 non-null  object 
 5   average_rating     12689 non-null  float64
 6   num_votes          12689 non-null  int64  
 7   persons_name       12689 non-null  object 
 8   persons_job        12689 non-null  object 
 9   title_norm         12689 non-null  object 
 10  id                 12689 non-null  int64  
 11  release_date       12689 non-null  object 
 12  movie              12689 non-null  object 
 13  production_budget  12689 non-null  object 
 14  domestic_gross     12689 non-null  object 
 15  worldwide_gross    12689 non-null  object 
dtypes: float64(1), int64(3

In [50]:
len(df_movie_info_budget['title_norm'].unique())

1348

### Now I am going to normalize the production_budget, domestic_gross, and worldwide_gross to convert them to integers

In [65]:
x = '$900,000,000'
x.replace(',', '').strip('$')

'900000000'

In [71]:
# list for the columns to be changed
columns = ['production_budget', 'domestic_gross', 'worldwide_gross']

# for loop that will go into each column I want to change and
# replace, strip, and convert it to a int
for column in columns:
    df_movie_info_budget[column] = df_movie_info_budget[column].str.replace(',','').str.strip('$').astype(int)

AttributeError: Can only use .str accessor with string values!

In [256]:
df_movie_info_budget.head(20)

,movie_id,primary_title,original_title,start_year,genres,average_rating,num_votes,persons_name,persons_job,title_norm,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Johnny Depp,actor,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110
1,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Helena Bonham Carter,actress,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110
2,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Tim Burton,director,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110
3,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Anne Hathaway,actress,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110
4,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Joe Roth,producer,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110
5,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Lewis Carroll,writer,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110
6,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Jennifer Todd,producer,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110
7,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Suzanne Todd,producer,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110
8,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Linda Woolverton,writer,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110
9,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Mia Wasikowska,actress,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110


### Now I want to get a list of all the unique genres in the genre column

In [150]:
df_movie_info_budget['genres'] = df_movie_info_budget['genres'].str.replace('Musical','Singing')

In [151]:
# turn the 'genres' column into a list
genres = list(df_movie_info_budget['genres'].unique())

In [152]:
# turn it into a list of a set of the genres list
genres = list(set(genres))

In [153]:
genres

['Singing',
 'Action,Biography,Drama',
 'Comedy,Drama,Fantasy',
 'Biography,Comedy,Drama',
 'Biography,Drama,Family',
 'Horror,Mystery,Sci-Fi',
 'Biography,Family,Sport',
 'Comedy,Drama,Singing',
 'Adventure,Family,Sci-Fi',
 'Drama,Sci-Fi,Thriller',
 'Drama,Mystery,War',
 'Comedy,Family',
 'Action,Crime,Horror',
 'Comedy,Crime,Thriller',
 'Comedy,Music,War',
 'Crime,Drama,Horror',
 'Action,Drama,Family',
 'Drama,History,War',
 'Biography,Drama,Music',
 'Comedy,Horror,Romance',
 'Action,Sci-Fi',
 'Drama,Fantasy,Music',
 'Drama,Fantasy,Mystery',
 'Comedy,Horror,Sci-Fi',
 'Action,Adventure,Fantasy',
 'Drama,Mystery,Romance',
 'Comedy,Crime,Romance',
 'Comedy,Fantasy,Horror',
 'Crime,Drama,Thriller',
 'Drama,Romance',
 'Action,Biography,Crime',
 'Comedy,Documentary',
 'Drama,Horror,Sci-Fi',
 'Animation',
 'Crime,Horror,Mystery',
 'Action,Biography,Documentary',
 'Action,Adventure',
 'Action,Fantasy,Western',
 'Action,Crime',
 'Action,Drama',
 'Comedy,Music',
 'Comedy,Drama,Romance',
 'Acti

In [154]:
'Action,Biography,Drama'.split(',')

['Action', 'Biography', 'Drama']

In [155]:
# for loop to take each string item in genre and split it by ','
genres_split = []

for string in genres:
    string = string.split(',')
    genres_split.append(string)
genres_split

[['Singing'],
 ['Action', 'Biography', 'Drama'],
 ['Comedy', 'Drama', 'Fantasy'],
 ['Biography', 'Comedy', 'Drama'],
 ['Biography', 'Drama', 'Family'],
 ['Horror', 'Mystery', 'Sci-Fi'],
 ['Biography', 'Family', 'Sport'],
 ['Comedy', 'Drama', 'Singing'],
 ['Adventure', 'Family', 'Sci-Fi'],
 ['Drama', 'Sci-Fi', 'Thriller'],
 ['Drama', 'Mystery', 'War'],
 ['Comedy', 'Family'],
 ['Action', 'Crime', 'Horror'],
 ['Comedy', 'Crime', 'Thriller'],
 ['Comedy', 'Music', 'War'],
 ['Crime', 'Drama', 'Horror'],
 ['Action', 'Drama', 'Family'],
 ['Drama', 'History', 'War'],
 ['Biography', 'Drama', 'Music'],
 ['Comedy', 'Horror', 'Romance'],
 ['Action', 'Sci-Fi'],
 ['Drama', 'Fantasy', 'Music'],
 ['Drama', 'Fantasy', 'Mystery'],
 ['Comedy', 'Horror', 'Sci-Fi'],
 ['Action', 'Adventure', 'Fantasy'],
 ['Drama', 'Mystery', 'Romance'],
 ['Comedy', 'Crime', 'Romance'],
 ['Comedy', 'Fantasy', 'Horror'],
 ['Crime', 'Drama', 'Thriller'],
 ['Drama', 'Romance'],
 ['Action', 'Biography', 'Crime'],
 ['Comedy', 'Doc

In [156]:
# create a set to add all the unique genres too
unique_genres = set()

# for loop to take each item in the genres_split and 
# add it to the unique_genres list. End result should be 
# list of all unique genres that appear in the 'genres' column
for item in genres_split:
    for genre in item:
        unique_genres.add(genre)
        
unique_genres = list(unique_genres)       
unique_genres

['Singing',
 'History',
 'Family',
 'Action',
 'Comedy',
 'Western',
 'Mystery',
 'Documentary',
 'Fantasy',
 'War',
 'Sport',
 'Sci-Fi',
 'Horror',
 'Animation',
 'Drama',
 'Biography',
 'Adventure',
 'Music',
 'Romance',
 'Thriller',
 'Crime']

In [157]:
len(unique_genres)

21

In [158]:
# Test conditional
'Horror' in 'Action,Crime,Horror'

True

### Now I have a list of all the data frames I'll create a new df_genres to contain the movie_id, title_norm, and genres columns

In [227]:
df_movie_info_budget.head()

,movie_id,primary_title,original_title,start_year,genres,average_rating,num_votes,persons_name,persons_job,title_norm,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Johnny Depp,actor,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110
1,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Helena Bonham Carter,actress,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110
2,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Tim Burton,director,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110
3,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Anne Hathaway,actress,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110
4,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Joe Roth,producer,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110


In [298]:
relevant_columns = ['movie_id', 'title_norm', 'genres']
df_genres = df_movie_info_budget[relevant_columns]

In [299]:
df_genres.head(20)

,movie_id,title_norm,genres
0,tt1014759,alice in wonderland 2010,"Adventure,Family,Fantasy"
1,tt1014759,alice in wonderland 2010,"Adventure,Family,Fantasy"
2,tt1014759,alice in wonderland 2010,"Adventure,Family,Fantasy"
3,tt1014759,alice in wonderland 2010,"Adventure,Family,Fantasy"
4,tt1014759,alice in wonderland 2010,"Adventure,Family,Fantasy"
5,tt1014759,alice in wonderland 2010,"Adventure,Family,Fantasy"
6,tt1014759,alice in wonderland 2010,"Adventure,Family,Fantasy"
7,tt1014759,alice in wonderland 2010,"Adventure,Family,Fantasy"
8,tt1014759,alice in wonderland 2010,"Adventure,Family,Fantasy"
9,tt1014759,alice in wonderland 2010,"Adventure,Family,Fantasy"


#### Now I want to create new columns with the list of genres and add a 1 if that movie contains that genre and 0 if it doesn't

In [300]:
# for loop to create new columns in df_genre
for genre in unique_genres:
    df_genres[genre] = np.where(df_genres['genres'].str.contains(genre), 1, 0)
#     if genre in df_genres[genres]: 1 else: 0

<ipython-input-300-4338c4b62e2d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_genres[genre] = np.where(df_genres['genres'].str.contains(genre), 1, 0)


In [301]:
df_genres

,movie_id,title_norm,genres,Singing,History,Family,Action,Comedy,Western,Mystery,...,Sci-Fi,Horror,Animation,Drama,Biography,Adventure,Music,Romance,Thriller,Crime
0,tt1014759,alice in wonderland 2010,"Adventure,Family,Fantasy",0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,tt1014759,alice in wonderland 2010,"Adventure,Family,Fantasy",0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,tt1014759,alice in wonderland 2010,"Adventure,Family,Fantasy",0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,tt1014759,alice in wonderland 2010,"Adventure,Family,Fantasy",0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,tt1014759,alice in wonderland 2010,"Adventure,Family,Fantasy",0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20410,tt6139732,aladdin 2019,"Adventure,Comedy,Family",0,0,1,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
20411,tt6139732,aladdin 2019,"Adventure,Comedy,Family",0,0,1,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
20412,tt6139732,aladdin 2019,"Adventure,Comedy,Family",0,0,1,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
20413,tt6139732,aladdin 2019,"Adventure,Comedy,Family",0,0,1,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0


In [302]:
# sanity check. I'll add a new column that adds all the rows accross
df_genres['total_genre'] = df_genres.iloc[:,2:].sum(axis=1)

In [303]:
df_genres

,movie_id,title_norm,genres,Singing,History,Family,Action,Comedy,Western,Mystery,...,Horror,Animation,Drama,Biography,Adventure,Music,Romance,Thriller,Crime,total_genre
0,tt1014759,alice in wonderland 2010,"Adventure,Family,Fantasy",0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,3
1,tt1014759,alice in wonderland 2010,"Adventure,Family,Fantasy",0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,3
2,tt1014759,alice in wonderland 2010,"Adventure,Family,Fantasy",0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,3
3,tt1014759,alice in wonderland 2010,"Adventure,Family,Fantasy",0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,3
4,tt1014759,alice in wonderland 2010,"Adventure,Family,Fantasy",0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20410,tt6139732,aladdin 2019,"Adventure,Comedy,Family",0,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,3
20411,tt6139732,aladdin 2019,"Adventure,Comedy,Family",0,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,3
20412,tt6139732,aladdin 2019,"Adventure,Comedy,Family",0,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,3
20413,tt6139732,aladdin 2019,"Adventure,Comedy,Family",0,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,3


In [304]:
df_genres['total_genre'].value_counts()

3    8561
2    2967
1    1161
Name: total_genre, dtype: int64

In [305]:
df_genres.loc[df_genres['total_genre'] == 4]

,movie_id,title_norm,genres,Singing,History,Family,Action,Comedy,Western,Mystery,...,Horror,Animation,Drama,Biography,Adventure,Music,Romance,Thriller,Crime,total_genre


In [306]:
df_genres.loc[df_genres['Singing'] == 1]

,movie_id,title_norm,genres,Singing,History,Family,Action,Comedy,Western,Mystery,...,Horror,Animation,Drama,Biography,Adventure,Music,Romance,Thriller,Crime,total_genre
30,tt2049386,alice in wonderland 2010,"Fantasy,Singing",1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
31,tt2049386,alice in wonderland 2010,"Fantasy,Singing",1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
32,tt2049386,alice in wonderland 2010,"Fantasy,Singing",1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
33,tt2049386,alice in wonderland 2010,"Fantasy,Singing",1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
34,tt2049386,alice in wonderland 2010,"Fantasy,Singing",1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17074,tt1485796,the greatest showman 2017,"Biography,Drama,Singing",1,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,3
17075,tt1485796,the greatest showman 2017,"Biography,Drama,Singing",1,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,3
17076,tt1485796,the greatest showman 2017,"Biography,Drama,Singing",1,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,3
17077,tt1485796,the greatest showman 2017,"Biography,Drama,Singing",1,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,3


#### Everything looks great now. next step is to drop total_genre column and transform this data from wide to long format

In [307]:
df_genres.drop(columns='total_genre', inplace=True)

In [311]:
list(df_genres.iloc[:,3:].columns)

['Singing',
 'History',
 'Family',
 'Action',
 'Comedy',
 'Western',
 'Mystery',
 'Documentary',
 'Fantasy',
 'War',
 'Sport',
 'Sci-Fi',
 'Horror',
 'Animation',
 'Drama',
 'Biography',
 'Adventure',
 'Music',
 'Romance',
 'Thriller',
 'Crime']

In [315]:
# Now to change to long format from wide
cols_to_keep = ['movie_id', 'title_norm', 'genres']
cols_to_move = list(df_genres.iloc[:,3:].columns)

df_genre_long = pd.melt(df_genres, 
                        id_vars=cols_to_keep, 
                        value_vars=cols_to_move, 
                        var_name='genre',
                        value_name='genre_true_or_false')

In [317]:
df_genre_long.head(15)

,movie_id,title_norm,genres,genre,genre_true_or_false
0,tt1014759,alice in wonderland 2010,"Adventure,Family,Fantasy",Singing,0
1,tt1014759,alice in wonderland 2010,"Adventure,Family,Fantasy",Singing,0
2,tt1014759,alice in wonderland 2010,"Adventure,Family,Fantasy",Singing,0
3,tt1014759,alice in wonderland 2010,"Adventure,Family,Fantasy",Singing,0
4,tt1014759,alice in wonderland 2010,"Adventure,Family,Fantasy",Singing,0
5,tt1014759,alice in wonderland 2010,"Adventure,Family,Fantasy",Singing,0
6,tt1014759,alice in wonderland 2010,"Adventure,Family,Fantasy",Singing,0
7,tt1014759,alice in wonderland 2010,"Adventure,Family,Fantasy",Singing,0
8,tt1014759,alice in wonderland 2010,"Adventure,Family,Fantasy",Singing,0
9,tt1014759,alice in wonderland 2010,"Adventure,Family,Fantasy",Singing,0


#### Now the next step is to drop all the rows with 0 in the genre_true_or_false column

In [318]:
df_genre_long = df_genre_long.loc[df_genre_long['genre_true_or_false'] == 1]

In [319]:
# now I'll drop duplicate values
df_genre_long.drop_duplicates(inplace=True)

In [320]:
# sort values by title_norm column
df_genre_long.sort_values(by='title_norm', inplace=True)

In [321]:
# drop genres and genre_true_or_false columns because they are unnecessary
df_genre_long.drop(columns=['genres', 'genre_true_or_false'], inplace=True)

In [322]:
df_genre_long

,movie_id,title_norm,genre
186317,tt3526286,#horror 2015,Drama
262451,tt3526286,#horror 2015,Crime
160939,tt3526286,#horror 2015,Horror
186551,tt1179933,10 cloverfield lane 2016,Drama
85039,tt1179933,10 cloverfield lane 2016,Mystery
...,...,...,...
53011,tt1222817,zookeeper 2011,Comedy
59748,tt1608290,zoolander 2 2016,Comedy
60472,tt2948356,zootopia 2016,Comedy
212740,tt2948356,zootopia 2016,Adventure


### Now that we have a long format genre table we can merge it with the movie info dataframe

In [323]:
df_movie_info_budget.head()

,movie_id,primary_title,original_title,start_year,genres,average_rating,num_votes,persons_name,persons_job,title_norm,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Johnny Depp,actor,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110
1,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Helena Bonham Carter,actress,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110
2,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Tim Burton,director,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110
3,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Anne Hathaway,actress,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110
4,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Joe Roth,producer,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110


In [324]:
df_genre_long.head()

,movie_id,title_norm,genre
186317,tt3526286,#horror 2015,Drama
262451,tt3526286,#horror 2015,Crime
160939,tt3526286,#horror 2015,Horror
186551,tt1179933,10 cloverfield lane 2016,Drama
85039,tt1179933,10 cloverfield lane 2016,Mystery


In [332]:
df_movie_genre_long =  df_movie_info_budget.merge(df_genre_long, 
                                                    how='inner',
                                                    suffixes=('_movie_info', '_genre_long'),
                                                    on='movie_id')

In [333]:
df_movie_genre_long.head()

,movie_id,primary_title,original_title,start_year,genres,average_rating,num_votes,persons_name,persons_job,title_norm_movie_info,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,title_norm_genre_long,genre
0,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Johnny Depp,actor,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110,alice in wonderland 2010,Family
1,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Johnny Depp,actor,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110,alice in wonderland 2010,Fantasy
2,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Johnny Depp,actor,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110,alice in wonderland 2010,Adventure
3,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Helena Bonham Carter,actress,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110,alice in wonderland 2010,Family
4,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Helena Bonham Carter,actress,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110,alice in wonderland 2010,Fantasy


In [334]:
# check for duplicates
df_movie_genre_long.loc[df_movie_genre_long.duplicated()]

,movie_id,primary_title,original_title,start_year,genres,average_rating,num_votes,persons_name,persons_job,title_norm_movie_info,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,title_norm_genre_long,genre


In [335]:
# check out the duplicates more
df_movie_genre_long.loc[df_movie_genre_long['primary_title'] == 'Aladdin']

,movie_id,primary_title,original_title,start_year,genres,average_rating,num_votes,persons_name,persons_job,title_norm_movie_info,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,title_norm_genre_long,genre
32751,tt6139732,Aladdin,Aladdin,2019,"Adventure,Comedy,Family",7.4,57549,Will Smith,actor,aladdin 2019,81,"May 24, 2019",Aladdin,182000000,246734314,619234314,aladdin 2019,Comedy
32752,tt6139732,Aladdin,Aladdin,2019,"Adventure,Comedy,Family",7.4,57549,Will Smith,actor,aladdin 2019,81,"May 24, 2019",Aladdin,182000000,246734314,619234314,aladdin 2019,Adventure
32753,tt6139732,Aladdin,Aladdin,2019,"Adventure,Comedy,Family",7.4,57549,Will Smith,actor,aladdin 2019,81,"May 24, 2019",Aladdin,182000000,246734314,619234314,aladdin 2019,Family
32754,tt6139732,Aladdin,Aladdin,2019,"Adventure,Comedy,Family",7.4,57549,Guy Ritchie,director,aladdin 2019,81,"May 24, 2019",Aladdin,182000000,246734314,619234314,aladdin 2019,Comedy
32755,tt6139732,Aladdin,Aladdin,2019,"Adventure,Comedy,Family",7.4,57549,Guy Ritchie,director,aladdin 2019,81,"May 24, 2019",Aladdin,182000000,246734314,619234314,aladdin 2019,Adventure
32756,tt6139732,Aladdin,Aladdin,2019,"Adventure,Comedy,Family",7.4,57549,Guy Ritchie,director,aladdin 2019,81,"May 24, 2019",Aladdin,182000000,246734314,619234314,aladdin 2019,Family
32757,tt6139732,Aladdin,Aladdin,2019,"Adventure,Comedy,Family",7.4,57549,John August,writer,aladdin 2019,81,"May 24, 2019",Aladdin,182000000,246734314,619234314,aladdin 2019,Comedy
32758,tt6139732,Aladdin,Aladdin,2019,"Adventure,Comedy,Family",7.4,57549,John August,writer,aladdin 2019,81,"May 24, 2019",Aladdin,182000000,246734314,619234314,aladdin 2019,Adventure
32759,tt6139732,Aladdin,Aladdin,2019,"Adventure,Comedy,Family",7.4,57549,John August,writer,aladdin 2019,81,"May 24, 2019",Aladdin,182000000,246734314,619234314,aladdin 2019,Family
32760,tt6139732,Aladdin,Aladdin,2019,"Adventure,Comedy,Family",7.4,57549,Alan Menken,composer,aladdin 2019,81,"May 24, 2019",Aladdin,182000000,246734314,619234314,aladdin 2019,Comedy


In [336]:
# remove duplicates
df_movie_genre_long.drop_duplicates(inplace=True)

In [337]:
df_movie_genre_long.head()

,movie_id,primary_title,original_title,start_year,genres,average_rating,num_votes,persons_name,persons_job,title_norm_movie_info,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,title_norm_genre_long,genre
0,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Johnny Depp,actor,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110,alice in wonderland 2010,Family
1,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Johnny Depp,actor,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110,alice in wonderland 2010,Fantasy
2,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Johnny Depp,actor,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110,alice in wonderland 2010,Adventure
3,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Helena Bonham Carter,actress,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110,alice in wonderland 2010,Family
4,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Helena Bonham Carter,actress,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110,alice in wonderland 2010,Fantasy


In [338]:
# check out final data frame
df_movie_genre_long.head(65)

,movie_id,primary_title,original_title,start_year,genres,average_rating,num_votes,persons_name,persons_job,title_norm_movie_info,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,title_norm_genre_long,genre
0,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Johnny Depp,actor,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110,alice in wonderland 2010,Family
1,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Johnny Depp,actor,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110,alice in wonderland 2010,Fantasy
2,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Johnny Depp,actor,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110,alice in wonderland 2010,Adventure
3,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Helena Bonham Carter,actress,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110,alice in wonderland 2010,Family
4,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Helena Bonham Carter,actress,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110,alice in wonderland 2010,Fantasy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,tt1294226,The Last Song,The Last Song,2010,"Drama,Music,Romance",6.0,74914,Miley Cyrus,actress,the last song 2010,15,"Mar 31, 2010",The Last Song,20000000,62950384,92678948,the last song 2010,Drama
61,tt1294226,The Last Song,The Last Song,2010,"Drama,Music,Romance",6.0,74914,Miley Cyrus,actress,the last song 2010,15,"Mar 31, 2010",The Last Song,20000000,62950384,92678948,the last song 2010,Romance
62,tt1294226,The Last Song,The Last Song,2010,"Drama,Music,Romance",6.0,74914,Julie Anne Robinson,director,the last song 2010,15,"Mar 31, 2010",The Last Song,20000000,62950384,92678948,the last song 2010,Music
63,tt1294226,The Last Song,The Last Song,2010,"Drama,Music,Romance",6.0,74914,Julie Anne Robinson,director,the last song 2010,15,"Mar 31, 2010",The Last Song,20000000,62950384,92678948,the last song 2010,Drama


In [339]:
df_movie_genre_long.shape

(32778, 18)

In [340]:
df_movie_genre_long.loc[df_movie_genre_long['primary_title'] == 'Alice in Wonderland']

,movie_id,primary_title,original_title,start_year,genres,average_rating,num_votes,persons_name,persons_job,title_norm_movie_info,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,title_norm_genre_long,genre
0,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Johnny Depp,actor,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110,alice in wonderland 2010,Family
1,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Johnny Depp,actor,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110,alice in wonderland 2010,Fantasy
2,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Johnny Depp,actor,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110,alice in wonderland 2010,Adventure
3,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Helena Bonham Carter,actress,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110,alice in wonderland 2010,Family
4,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Helena Bonham Carter,actress,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110,alice in wonderland 2010,Fantasy
5,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Helena Bonham Carter,actress,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110,alice in wonderland 2010,Adventure
6,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Tim Burton,director,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110,alice in wonderland 2010,Family
7,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Tim Burton,director,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110,alice in wonderland 2010,Fantasy
8,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Tim Burton,director,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110,alice in wonderland 2010,Adventure
9,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Anne Hathaway,actress,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110,alice in wonderland 2010,Family


# Data Preparation and Cleaning is done for the most part

## Here are the two data frames we are left with

In [343]:
df_movie_genre_long

,movie_id,primary_title,original_title,start_year,genres,average_rating,num_votes,persons_name,persons_job,title_norm_movie_info,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,title_norm_genre_long,genre
0,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Johnny Depp,actor,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110,alice in wonderland 2010,Family
1,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Johnny Depp,actor,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110,alice in wonderland 2010,Fantasy
2,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Johnny Depp,actor,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110,alice in wonderland 2010,Adventure
3,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Helena Bonham Carter,actress,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110,alice in wonderland 2010,Family
4,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Helena Bonham Carter,actress,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110,alice in wonderland 2010,Fantasy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32773,tt6139732,Aladdin,Aladdin,2019,"Adventure,Comedy,Family",7.4,57549,Naomi Scott,actress,aladdin 2019,81,"May 24, 2019",Aladdin,182000000,246734314,619234314,aladdin 2019,Adventure
32774,tt6139732,Aladdin,Aladdin,2019,"Adventure,Comedy,Family",7.4,57549,Naomi Scott,actress,aladdin 2019,81,"May 24, 2019",Aladdin,182000000,246734314,619234314,aladdin 2019,Family
32775,tt6139732,Aladdin,Aladdin,2019,"Adventure,Comedy,Family",7.4,57549,Mena Massoud,actor,aladdin 2019,81,"May 24, 2019",Aladdin,182000000,246734314,619234314,aladdin 2019,Comedy
32776,tt6139732,Aladdin,Aladdin,2019,"Adventure,Comedy,Family",7.4,57549,Mena Massoud,actor,aladdin 2019,81,"May 24, 2019",Aladdin,182000000,246734314,619234314,aladdin 2019,Adventure


In [344]:
df_movie_info_budget

,movie_id,primary_title,original_title,start_year,genres,average_rating,num_votes,persons_name,persons_job,title_norm,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Johnny Depp,actor,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110
1,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Helena Bonham Carter,actress,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110
2,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Tim Burton,director,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110
3,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Anne Hathaway,actress,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110
4,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Joe Roth,producer,alice in wonderland 2010,51,"Mar 5, 2010",Alice in Wonderland,200000000,334191110,1025491110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20410,tt6139732,Aladdin,Aladdin,2019,"Adventure,Comedy,Family",7.4,57549,Dan Lin,producer,aladdin 2019,81,"May 24, 2019",Aladdin,182000000,246734314,619234314
20411,tt6139732,Aladdin,Aladdin,2019,"Adventure,Comedy,Family",7.4,57549,Jonathan Eirich,producer,aladdin 2019,81,"May 24, 2019",Aladdin,182000000,246734314,619234314
20412,tt6139732,Aladdin,Aladdin,2019,"Adventure,Comedy,Family",7.4,57549,Marwan Kenzari,actor,aladdin 2019,81,"May 24, 2019",Aladdin,182000000,246734314,619234314
20413,tt6139732,Aladdin,Aladdin,2019,"Adventure,Comedy,Family",7.4,57549,Naomi Scott,actress,aladdin 2019,81,"May 24, 2019",Aladdin,182000000,246734314,619234314
